## Project: Gaussian discriminant analysis, Naive bayes and Logistic regression implementation

In [1]:
import pandas as pd
import numpy as np
import numpy.linalg as m
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
data=pd.read_table('drugLibTest_raw.tsv')

In [3]:
data.head()

,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,1366,biaxin,9,Considerably Effective,Mild Side Effects,sinus infection,The antibiotic may have destroyed bacteria cau...,"Some back pain, some nauseau.",Took the antibiotics for 14 days. Sinus infect...
1,3724,lamictal,9,Highly Effective,Mild Side Effects,bipolar disorder,Lamictal stabilized my serious mood swings. On...,"Drowsiness, a bit of mental numbness. If you t...",Severe mood swings between hypomania and depre...
2,3824,depakene,4,Moderately Effective,Severe Side Effects,bipolar disorder,Initial benefits were comparable to the brand ...,"Depakene has a very thin coating, which caused...",Depakote was prescribed to me by a Kaiser psyc...
3,969,sarafem,10,Highly Effective,No Side Effects,bi-polar / anxiety,It controlls my mood swings. It helps me think...,I didnt really notice any side effects.,This drug may not be for everyone but its wond...
4,696,accutane,10,Highly Effective,Mild Side Effects,nodular acne,Within one week of treatment superficial acne ...,Side effects included moderate to severe dry s...,Drug was taken in gelatin tablet at 0.5 mg per...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 9 columns):
Unnamed: 0           1036 non-null int64
urlDrugName          1036 non-null object
rating               1036 non-null int64
effectiveness        1036 non-null object
sideEffects          1036 non-null object
condition            1036 non-null object
benefitsReview       1036 non-null object
sideEffectsReview    1036 non-null object
commentsReview       1036 non-null object
dtypes: int64(2), object(7)
memory usage: 72.9+ KB


### Conversion of features to binary numbers

In [5]:
y = 1*(data['rating']>5)
df = data.drop(['Unnamed: 0','rating'],axis=1)

def tovec(text):
    # create the transform
    vectorizer = CountVectorizer(binary=True,stop_words='english',max_features=90)
    # tokenize and build vocab
    vectorizer.fit(text)
    # summarize
    k = vectorizer.vocabulary_
    # encode document
    vector = vectorizer.transform(text)
    # summarize encoded vector
    return vector.toarray()
data_new=df['urlDrugName']+' '+df['effectiveness']+' '+df['sideEffects']+' '+df['condition']+' '+df['benefitsReview']+' '+df['sideEffectsReview']+' '+df['commentsReview']
X = tovec(data_new)
df_features = pd.DataFrame(X)

In [6]:
df_features.head()

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1
3,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


### Calculus of Gaussian discriminant analysis parameters

In [7]:
def calculate_phi(Y):
    m = length(Y)
    return sum(x(x==1), Y)/m

def calculate_sigma(X, Y, mu0, mu1):
    m = length(Y)
    sqr_sum = zeros(size(X)[2], size(X)[2])
    for i in range(m):
        xi = X[i,:]
        yi = Y[i]
        sqr = sum((xi-mu1)*(xi-mu1).T(yi == 1)) 
        sqr_sum = sqr_sum + sqr
 
    return sqr_sum/m


def calculate_mu1(X, Y):
    m = length(Y)
    y_pos = count(x(x==1), Y)
    conditional_sum_x = zeros(X[1,:])
    for i in range(m):
        xi = X[i,:]
        yi = Y[i]
        conditional_sum_x = conditional_sum_x + sum(xi(yi == 1), zeros(xi))
    return (1/m)*conditional_sum_x/y_pos


def calculate_mu0(X, Y):
    m = length(Y)
    y_neg = count(x(x==0), Y)
    conditional_sum_x = zeros(X[1,:])
    for i in range(m):
        xi = X[i,:]
        yi = Y[i]
        conditional_sum_x = conditional_sum_x + sum(xi(yi == 0), zeros(xi))
    return (1/m)*conditional_sum_x/y_neg

def calculate_px_py(x, mu, sigma):
    n = 1
    pi = 3.14
    dim = len(mu)
    return ((1/(2*np.pi)^(dim/2)*np.sqrt(m.det(sigma)))*np.exp(-0.5*np.transpose(x-mu)*np.linalg.inv(sigma)*(x-mu)))

                                                  
def calculate_py(y, phi):
    return sum(phi(y==1), (1-phi))

def predict(self, X):
        y_pred = []
        for sample in X:
            h = sample.dot(self.w)
            y = 1 * (h < 0)
            y_pred.append(y)
        return y_pred

In [8]:
def cov(x, y):
    xbar, ybar = x.mean(), y.mean()
    return np.sum((x - xbar)*(y - ybar))/(len(x) - 1)

def calculate_covariance_matrix(X):
    return np.array([[cov(X[0], X[0]), cov(X[0], X[1])], \
                     [cov(X[1], X[0]), cov(X[1], X[1])]])

### Gaussian discriminant analysis class

In [9]:
class Gaussian_Discriminant_Analysis():
    
    def __init__(self):
        self.w = None

    def transform(self, X, y):
        self.fit(X, y)
        # Project data onto vector
        X_transform = X.dot(self.w)
        return X_transform

    def fit(self, X, y):
        # Separate data by class
        X1 = X[y == 0]
        X2 = X[y == 1]

        # Calculate the covariance matrices of the two datasets
        cov1 = calculate_covariance_matrix(X1)
        cov2 = calculate_covariance_matrix(X2)
        cov_tot = cov1 + cov2

        # Calculate the mean of the two datasets
        mean1 = X1.mean(0)
        mean2 = X2.mean(0)
        mean_diff = np.atleast_1d(mean1 - mean2) #convert to an array to 1D at least

        # Determine the vector which when X is projected onto it best separates the
        # data by class. w = (mean1 - mean2) / (cov1 + cov2)
        self.w = np.linalg.pinv(cov_tot).dot(mean_diff)

    def predict(self, X):
        y_pred = []
        for sample in X:
            h = sample.dot(self.w)
            y = 1 * (h < 0)
            y_pred.append(y)
        return y_pred
    
    def accuracy(self,X,y):
        y_pred = self.predict(X)
        accu=np.sum(y_pred == y)/len(y)
        return accu

### Test of Gaussian discriminant analysis on iris

In [10]:
data=datasets.load_iris()
x1=data.data[:,:2]
y1=(data.target!=0)*1

In [11]:
model=Gaussian_Discriminant_Analysis()

In [12]:
model.fit(x1,y1)

In [13]:
predictions=model.predict(x1)

In [14]:
model.accuracy(x1,y1)

0.6666666666666666

### Test of Gaussian discriminant analysis on our initial data

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=101)

In [16]:
model1=Gaussian_Discriminant_Analysis()

### Naive bayes implementation

In [17]:
class Naive_bayes:
    def __init__(self,X,y):
        self.X = X
        self.y = y
    def fit(self):
        data = self.X
        target = self.y
        x_1y_0=[]
        x_1y_1=[]
        y_new=[]
        count_3=0
        count_4=0
        for j in range(data.shape[1]):
            count=0
            count_2=0
            for i in range(data.shape[0]):
#                 print(data,target,i,j)
                if y[i]==0 and data[i][j]==1:
                    count +=1
                if y[i]==1 and data[i][j]==1:
                    count_2 +=1
                if j==0 and y[i]==1:
                    count_3+=1
                if j==0 and y[i]==0:
                    count_4+=1
            length_1=count_3
            length_0=count_4
            x_1y_0.append(count/length_0)
            x_1y_1.append(count_2/length_1)
        y_new.append(count_3/(length_0+length_1))
        return x_1y_0,x_1y_1,y_new

    def Bernouilli(self,x,phi):
        return (phi**x)*(1-phi)**(1-x)

    def predict(self,x):
        phi_x1_y0,phi_x1_y1,p = self.fit()
        pred = np.zeros(x.shape[0])
        for i in range(len(x)):
            prb1 = 1
            prb2 = 1
            for j in range(x.shape[1]):
                prb1*= self.Bernouilli(x[i,j],phi_x1_y0[j])
                prb2*= self.Bernouilli(x[i,j],phi_x1_y1[j])
            prb1 = prb1*(1-p[0])
            prb2 = prb2*(p[0])
            pred[i] = np.argmax([prb1,prb2])
        return pred

    def accuracy(self,test,pred):
        c=0
        for i in range(len(pred)):
            if(test.iloc[i]==pred[i]):
                c+=1
        c=c/len(pred)
        return c

### Test of Naive bayes

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=101)

In [19]:
model=Naive_bayes(X_train,y_train)

In [20]:
model.fit()

([0.08928571428571429,
  0.16666666666666666,
  0.10714285714285714,
  0.1488095238095238,
  0.125,
  0.05952380952380952,
  0.11904761904761904,
  0.15476190476190477,
  0.07142857142857142,
  0.32142857142857145,
  0.08928571428571429,
  0.16666666666666666,
  0.43452380952380953,
  0.23214285714285715,
  0.2619047619047619,
  0.27976190476190477,
  0.08928571428571429,
  0.16666666666666666,
  0.1130952380952381,
  0.20238095238095238,
  0.25,
  0.125,
  0.1130952380952381,
  0.9226190476190477,
  1.0,
  0.05952380952380952,
  0.05357142857142857,
  0.15476190476190477,
  0.23809523809523808,
  0.14285714285714285,
  0.16071428571428573,
  0.06547619047619048,
  0.08928571428571429,
  0.07738095238095238,
  0.1130952380952381,
  0.1130952380952381,
  0.05952380952380952,
  0.35714285714285715,
  0.08928571428571429,
  0.1130952380952381,
  0.09523809523809523,
  0.16071428571428573,
  0.10119047619047619,
  0.21428571428571427,
  0.10714285714285714,
  0.13690476190476192,
  0.22619

In [21]:
predictions = model.predict(X_test)

In [22]:
accuracy = model.accuracy(y_test,predictions)

In [23]:
print('The accuracy is: ',accuracy)

The accuracy is:  0.693050193050193


### Split and test on 60% 40% of the data

In [24]:
order = np.random.permutation(len(X))
portion = 40

"""
    Split your data into train and test using the order and permutation variable
"""
train_x = X[order[portion:]]
train_y = y[order[portion:]]

test_x= X[order[:portion]]
test_y= y[order[:portion]]

In [25]:
model=Naive_bayes(train_x,train_y)

In [26]:
model.fit()

([0.09764309764309764,
  0.13468013468013468,
  0.0707070707070707,
  0.13468013468013468,
  0.11784511784511785,
  0.07744107744107744,
  0.11447811447811448,
  0.16161616161616163,
  0.11447811447811448,
  0.3265993265993266,
  0.1111111111111111,
  0.16161616161616163,
  0.45791245791245794,
  0.22895622895622897,
  0.2053872053872054,
  0.31986531986531985,
  0.13804713804713806,
  0.21548821548821548,
  0.13468013468013468,
  0.18855218855218855,
  0.26936026936026936,
  0.09090909090909091,
  0.1919191919191919,
  0.936026936026936,
  1.0,
  0.06397306397306397,
  0.09764309764309764,
  0.12121212121212122,
  0.20202020202020202,
  0.13804713804713806,
  0.16498316498316498,
  0.10101010101010101,
  0.09090909090909091,
  0.12794612794612795,
  0.08754208754208755,
  0.10101010101010101,
  0.10774410774410774,
  0.3939393939393939,
  0.09427609427609428,
  0.12794612794612795,
  0.08080808080808081,
  0.15488215488215487,
  0.09427609427609428,
  0.20202020202020202,
  0.08080808

In [27]:
predictions = model.predict(X_test)

In [28]:
accuracy = model.accuracy(y_test,predictions)

In [29]:
print('The accuracy is: ',accuracy)

The accuracy is:  0.637065637065637


### Logistic regression implementation

In [30]:
class LogisticRegression():
    def __init__(self,lr,n_iters,weight,x,y):
        self.lr = lr
        self.n_iters = n_iters
        self.weight = weight
        self.x = x
        self.y = y
        
    def Sigmoid(self,z):
        sign = 1 / (1+np.exp(-z))
        return sign
    
    def Cost(self):
        yhat= self.x@self.weight
        cost = ((-self.y.T) @ np.log2(self.Sigmoid(yhat))) - ((1 - self.y).T @ np.log2(1 - self.Sigmoid(yhat)))
        #print(yhat.shape)
        return cost
    
    def Gradient(self):
        yhat= self.x@self.weight
        return self.x.T @ ((self.y) - self.Sigmoid(yhat))
    
    def fit(self):
        loss = np.zeros((self.n_iters))
        for i in range(self.n_iters):
            self.weight = self.weight + (self.lr * self.Gradient())
            
            loss[i] = self.Cost()
            
        return self.weight, loss
    
    
    def pred_prob(self,z):
            return self.Sigmoid(np.dot(z,self.weight))
    
    def predict(self,prediction):
    
         liste = []

         for elt in prediction:
                if elt<0.5:
                    liste.append(0)
                else:
                    liste.append(1)
                liste = np.array(liste)
                liste = liste.reshape(-1,1)
                return liste
    
    def hessian(self):
        yhat = self.x@self.weight
        omega = self.Sigmoid(yhat) @ (1 - self.Sigmoid(yhat)).T
        return ((-x.T) @ omega)@x -2.44641553e+05, LA.eigvals(((-x.T) @ omega)@x)

    def Newton(self):
        for i in range(self.n_iters):
            self.weight = self.weight - inv(self.hessian()[0])@self.Gradient()
        return self.weight
    
    def Good_pred(self,z):
        
        yhat=z@self.weight
        
        return np.where(self.Sigmoid(yhat)>0.5,1,0)
    
    
def accuracy(prediction,y):
    acc = y==prediction
    return np.sum(acc)/len(y)
        
def Sigmoid(x):
    sign = 1 / (1+np.exp(-x))
    return sign

def Good_pred(z, weight):
        
        yhat=z@weight
        
        return np.where(Sigmoid(yhat)>0.5,1,0)        

### Test of Logistic regression

In [31]:
data1=datasets.load_iris()
x1=data1.data[:,:2]
y1=(data1.target!=0)*1
y1=y1.reshape(-1,1)
theta = np.zeros((x1.shape[1],1))  

In [32]:
model=LogisticRegression(0.01,10000,theta,x1,y1)

In [33]:
result=model.fit()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log2
  from ipykernel import kernelapp as app
/usr/lib/python3/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in matmul
  from ipykernel import kernelapp as app


In [34]:
pred1= Good_pred(x1,result[0])
theta = np.zeros((x1.shape[1],1))

In [35]:
accuracy(pred1,y1)

0.9933333333333333

### Test logistic regression on our dataset

In [36]:
theta1 = np.zeros((X_train.shape[1],1))
model2=LogisticRegression(0.01,10000,theta1,X,y)

### Feature selection 

In [37]:
def forward_feature_selection(X, y, n_features=2):
    results = {'best':[]}
    count = 0
    feature_list = []
    while len(feature_list) < n_features:
        results[count] = []
        for i in range(len(X[0])):
            if i not in feature_list: 
                train_model_save_result(X, y, results[count])
        best_result = get_best_result(results[count])
        feature_list = best_result['features']
        results['best'].append(best_result)
        count+= 1
    compute_best_features(results, n_features)
    return results

def backward_feature_selection(X, y, n_features=2):
    results = {'best':[]}
    count = 0
    feature_list = range(len(X[0]))
    while len(feature_list) > 1:
        results[count] = []
        for i in range(len(feature_list)):
            idx = feature_list[0:i] + feature_list[i+1:]
            train_model_save_result(X, y, idx, results[count])
        best_result = get_best_result(results[count])
        feature_list = best_result['features']
        results['best'].append(best_result)
        count+= 1
    compute_best_features(results, n_features)
    return results

def compute_best_features(results, n_features):
    best_result = None
    for result in results['best']:
        if len(result['features']) <= n_features:
            if not best_result: 
                best_result = result
            elif best_result['score'] < result['score']: 
                best_result = result
    results['best_features'] = best_result['features']
    
def train_model_save_result(X, y, result_storage):
    #X_ = X[:, idx]
    result = 0
    model = Naive_bayes(X,y)
    model.fit()
    acc = accuracy(y, model.predict(X))
    result += acc
    print({'accuracy ' :acc})
    result_storage.append({'score': result})
    
def get_best_result(result_storage):
    best = None
    for model in result_storage:
        if not best: best = model
        elif model['score'] > best['score']:best = model
    return best

### Report

#### By observing the different accuracy of Gaussian discriminant analysis, Naive bayes and Logistic regression we can say that the Logistic regression model is better to use on this data than the Gaussian discriminant analysis and Naive bayes

## Thank you!